In [ ]:
import pandas as pd
import os
import time
import subprocess
import json
from collections import defaultdict

TEMP_DIR = 'scratchpad'
PROCESSED_COMMENTS_DIR = 'processed_comments'
COMMENTS_CSV = f'{PROCESSED_COMMENTS_DIR}/comments.csv'

os.makedirs(TEMP_DIR, exist_ok=True)
os.makedirs(PROCESSED_COMMENTS_DIR, exist_ok=True)

def run_yt_dlp(video_id, max_comments=100):
    filepath = os.path.join(TEMP_DIR, f"{video_id}")
    command = [
        "yt-dlp",
        "--skip-download",
        "--write-comments",
        "--no-warnings",
        "--output", filepath,
        "--extractor-args", f"youtube:max_comments={max_comments}",
        f"https://www.youtube.com/watch?v={video_id}"
    ]
    subprocess.run(command, check=True, text=True)


def parse_comments(video_id):
    with open(os.path.join(TEMP_DIR, f'{video_id}.info.json'), "r", encoding="utf-8") as f:
        data = json.load(f)
    comments = data['comments']
    title = data['title']
    uploader = data['uploader_id']

    comment_dict = {}
    replies = defaultdict(list)
    for c in comments:
        if c.get("parent") and c["parent"] != "root":
            replies[c["parent"]].append({**c, "is_reply": True})
        else:
            comment_dict[c["id"]] = {**c, "replies": [], "is_reply": False}
    for pid, rep in replies.items():
        if pid in comment_dict:
            comment_dict[pid]["replies"] = rep
    return list(comment_dict.values()), title, uploader


def to_process(video_id):
    if not os.path.exists(COMMENTS_CSV):
        return True
    df = pd.read_csv(COMMENTS_CSV)
    return video_id not in df['id'].astype(str).values


# Main loop
df_url_details = pd.read_csv('URLs/url_channel_id.csv')
data_res = []

for video_id in df_url_details.id.astype(str):
    if not to_process(video_id):
        #print(f"⏩ Skipping already processed video ID: {video_id}")
        continue

    try:
        run_yt_dlp(video_id, max_comments=100)
        comments, title, uploader = parse_comments(video_id)

        data_res.append({
            'creator_id': uploader,
            'id': video_id,
            'title': title,
            'timestamp': str(time.time()),
            'comments': json.dumps(comments)
        })

        # Append to CSV immediately to avoid reprocessing on crash
        df = pd.DataFrame(data_res)
        if os.path.exists(COMMENTS_CSV):
            df.to_csv(COMMENTS_CSV, mode='a', index=False, header=False)
        else:
            df.to_csv(COMMENTS_CSV, index=False)
        data_res.clear()

    except Exception as e:
        print(f"❌ Failed to process video ID: {video_id}")
        print(e)

[youtube] Extracting URL: https://www.youtube.com/watch?v=4rvFm18eyfU
[youtube] 4rvFm18eyfU: Downloading webpage
[youtube] 4rvFm18eyfU: Downloading tv client config
[youtube] 4rvFm18eyfU: Downloading tv player API JSON
[youtube] 4rvFm18eyfU: Downloading ios player API JSON
[youtube] 4rvFm18eyfU: Downloading m3u8 information
[youtube] Downloading comment section API JSON
[info] 4rvFm18eyfU: Downloading 1 format(s): 616+251
[info] There are no subtitles for the requested languages
Deleting existing file scratchpad/4rvFm18eyfU.webp
[info] Downloading video thumbnail 44 ...
[info] Writing video thumbnail 44 to: scratchpad/4rvFm18eyfU.webp
[info] Writing video metadata as JSON to: scratchpad/4rvFm18eyfU.info.json
❌ Failed to process video ID: 4rvFm18eyfU
'comments'
[youtube] Extracting URL: https://www.youtube.com/watch?v=2-WBcZeTY8k
[youtube] 2-WBcZeTY8k: Downloading webpage
[youtube] 2-WBcZeTY8k: Downloading tv client config
[youtube] 2-WBcZeTY8k: Downloading tv player API JSON
[youtube] 



ERROR: Did not get any data blocks


[youtube] Extracting URL: https://www.youtube.com/watch?v=QhevOHvbPMM
[youtube] QhevOHvbPMM: Downloading webpage
[youtube] QhevOHvbPMM: Downloading tv client config
[youtube] QhevOHvbPMM: Downloading tv player API JSON
[youtube] QhevOHvbPMM: Downloading ios player API JSON
[youtube] QhevOHvbPMM: Downloading m3u8 information
[info] QhevOHvbPMM: Downloading subtitles: en-IN
[youtube] Downloading comment section API JSON
[youtube] Downloading ~4882 comments
[youtube] Sorting comments by newest first
[youtube] Downloading comment API JSON page 1 (0/~4882)
[youtube]     Downloading comment API JSON reply thread 1 (1/~4882)
[youtube]        Downloading comment replies API JSON page 1 (11/~4882)
[youtube] Downloading comment API JSON page 2 (69/~4882)
[youtube] Downloading comment API JSON page 3 (89/~4882)
[youtube] Extracted 100 comments
[info] QhevOHvbPMM: Downloading 1 format(s): 616+251
[info] Writing video subtitles to: scratchpad/QhevOHvbPMM.en-IN.vtt

Deleting existing file scratchpad



ERROR: Did not get any data blocks


[youtube] Extracting URL: https://www.youtube.com/watch?v=uCk5k6t8Afo
[youtube] uCk5k6t8Afo: Downloading webpage
[youtube] uCk5k6t8Afo: Downloading tv client config
[youtube] uCk5k6t8Afo: Downloading tv player API JSON
[youtube] uCk5k6t8Afo: Downloading ios player API JSON
[youtube] uCk5k6t8Afo: Downloading m3u8 information
[info] uCk5k6t8Afo: Downloading subtitles: en-IN
[youtube] Downloading comment section API JSON
[youtube] Downloading ~8337 comments
[youtube] Sorting comments by newest first
[youtube] Downloading comment API JSON page 1 (0/~8337)
[youtube]     Downloading comment API JSON reply thread 1 (19/~8337)
[youtube] Downloading comment API JSON page 2 (21/~8337)
[youtube] Downloading comment API JSON page 3 (41/~8337)
[youtube]     Downloading comment API JSON reply thread 1 (42/~8337)
[youtube]     Downloading comment API JSON reply thread 2 (50/~8337)
[youtube]     Downloading comment API JSON reply thread 3 (58/~8337)
[youtube] Downloading comment API JSON page 4 (65/~8

...

❌ Failed to process video ID: LUs4luvC4JQ
Command '['yt-dlp', '--skip-download', '--write-comments', '--no-warnings', '--output', 'scratchpad/LUs4luvC4JQ', '--extractor-args', 'youtube:max_comments=100', 'https://www.youtube.com/watch?v=LUs4luvC4JQ']' returned non-zero exit status 1.
[youtube] Extracting URL: https://www.youtube.com/watch?v=a5YpJqSTqZ4
[youtube] a5YpJqSTqZ4: Downloading webpage
[youtube] a5YpJqSTqZ4: Downloading tv client config
[youtube] a5YpJqSTqZ4: Downloading tv player API JSON
[youtube] a5YpJqSTqZ4: Downloading ios player API JSON
[youtube] a5YpJqSTqZ4: Downloading m3u8 information
[info] a5YpJqSTqZ4: Downloading subtitles: en-IN
[youtube] Downloading comment section API JSON
[youtube] Downloading ~4290 comments
[youtube] Sorting comments by newest first
[youtube] Downloading comment API JSON page 1 (0/~4290)
[youtube]     Downloading comment API JSON reply thread 1 (1/~4290)
[youtube]        Downloading comment replies API JSON page 1 (11/~4290)
[youtube]        



ERROR: Did not get any data blocks


[youtube] Extracting URL: https://www.youtube.com/watch?v=HsBDTHGW6Yc
[youtube] HsBDTHGW6Yc: Downloading webpage
[youtube] HsBDTHGW6Yc: Downloading tv client config
[youtube] HsBDTHGW6Yc: Downloading tv player API JSON
[youtube] HsBDTHGW6Yc: Downloading ios player API JSON
[youtube] HsBDTHGW6Yc: Downloading m3u8 information
[info] HsBDTHGW6Yc: Downloading subtitles: en-IN
[youtube] Downloading comment section API JSON
[youtube] Downloading ~7856 comments
[youtube] Sorting comments by newest first
[youtube] Downloading comment API JSON page 1 (0/~7856)
[youtube]     Downloading comment API JSON reply thread 1 (1/~7856)
[youtube]     Downloading comment API JSON reply thread 2 (18/~7856)
[youtube] Downloading comment API JSON page 2 (29/~7856)
[youtube] Downloading comment API JSON page 3 (49/~7856)
[youtube] Downloading comment API JSON page 4 (69/~7856)
[youtube]     Downloading comment API JSON reply thread 1 (85/~7856)
[youtube] Downloading comment API JSON page 5 (90/~7856)
[youtube



ERROR: Did not get any data blocks


❌ Failed to process video ID: HsBDTHGW6Yc
Command '['yt-dlp', '--skip-download', '--write-comments', '--no-warnings', '--output', 'scratchpad/HsBDTHGW6Yc', '--extractor-args', 'youtube:max_comments=100', 'https://www.youtube.com/watch?v=HsBDTHGW6Yc']' returned non-zero exit status 1.
[youtube] Extracting URL: https://www.youtube.com/watch?v=OCxghFpX-r4
[youtube] OCxghFpX-r4: Downloading webpage
[youtube] OCxghFpX-r4: Downloading tv client config
[youtube] OCxghFpX-r4: Downloading tv player API JSON
[youtube] OCxghFpX-r4: Downloading ios player API JSON
[youtube] OCxghFpX-r4: Downloading m3u8 information
[info] OCxghFpX-r4: Downloading subtitles: live_chat
[youtube] Downloading comment section API JSON
[youtube] Downloading ~883 comments
[youtube] Sorting comments by newest first
[youtube] Downloading comment API JSON page 1 (0/~883)
[youtube] Downloading comment API JSON page 2 (20/~883)
[youtube]     Downloading comment API JSON reply thread 1 (27/~883)
[youtube] Downloading comment A

In [ ]:
df_test=pd.read_csv(f'{PROCESSED_COMMENTS_DIR}/comments.csv')
df_test['comments'] = df_test['comments'].apply(json.loads)

In [ ]:
df_test.shape

In [ ]:
df_test.tail()